In [7]:
import pymystem3, pymorphy2
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import seaborn as sns
import scipy.stats as st
from scipy.stats import norm
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, KBinsDiscretizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## Data Uploading

In [8]:
train = pd.read_csv('train.csv.zip', sep=';', index_col='id')
train.shape[0]

31063

In [9]:
test = pd.read_csv('test.csv.zip', sep=';');

In [10]:
train.head()

,text,target
id,,
0,<p>В молодой и дружный коллектив динамично раз...,0
1,<p><strong>Обязанности: </strong>- Прием входя...,0
2,<p><strong>Федеральная сеть аптек </strong>в с...,0
3,<strong>Обязанности:</strong> <ul> <li> <p>раз...,0
4,<p>Производителю сантехники тм Domani Spa треб...,1


In [11]:
train.shape, test.shape

((31063, 2), (31064, 2))

## Train Data Preprocessing

In [12]:
with open("stopwords.txt", encoding="utf-8") as file: 
        stop_words = file.read()

In [13]:
stop_words.split() # список собственных стоп-слов

['а',
 'будем',
 'будет',
 'будете',
 'будешь',
 'буду',
 'будут',
 'будучи',
 'будь',
 'будьте',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вами',
 'вас',
 'весь',
 'во',
 'вот',
 'все',
 'всё',
 'всего',
 'всей',
 'всем',
 'всём',
 'всеми',
 'всему',
 'всех',
 'всею',
 'всея',
 'всю',
 'вся',
 'вы',
 'да',
 'для',
 'до',
 'его',
 'едим',
 'едят',
 'ее',
 'её',
 'ей',
 'ел',
 'ела',
 'ем',
 'ему',
 'емъ',
 'если',
 'ест',
 'есть',
 'ешь',
 'еще',
 'ещё',
 'ею',
 'же',
 'за',
 'и',
 'из',
 'или',
 'им',
 'ими',
 'имъ',
 'их',
 'к',
 'как',
 'кем',
 'ко',
 'когда',
 'кого',
 'ком',
 'кому',
 'комья',
 'которая',
 'которого',
 'которое',
 'которой',
 'котором',
 'которому',
 'которою',
 'которую',
 'которые',
 'который',
 'которым',
 'которыми',
 'которых',
 'кто',
 'меня',
 'мне',
 'мной',
 'мною',
 'мог',
 'моги',
 'могите',
 'могла',
 'могли',
 'могло',
 'могу',
 'могут',
 'мое',
 'моё',
 'моего',
 'моей',
 'моем',
 'моём',
 'моему',
 'моею',
 'можем',
 'может'

In [14]:
# Pymorphy2
morph = pymorphy2.MorphAnalyzer() 
def normalizePymorphy2(text):
    #imp_POS = ['ADJF', 'ADJS', 'NOUN', 'VERB', 'PRTF', 'PRTS', 'GRND']
    imp_POS = ['ADJF', 'ADJS', 'NOUN']
    tokens = re.findall('[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+', text)
    with open("stopwords.txt", encoding="utf-8") as file: 
        stop_words = file.read()
    words = []
    for t in tokens:
        pv = morph.parse(t)
        if pv[0].tag.POS in imp_POS and pv[0].normal_form != 'быть' and \
        len(pv[0].normal_form) > 2 and pv[0].normal_form not in stop_words:
            words.append(pv[0].normal_form)
    return words    

In [15]:
# Функци нормализации, составления словаря и подсчета частотных слов
def getMostFrequentWordsFiltered(text):
    #imp_POS = ['ADJF', 'ADJS', 'NOUN', 'VERB', 'PRTF', 'PRTS', 'GRND']
    imp_POS = ['ADJF', 'ADJS', 'NOUN']
    morph = pymorphy2.MorphAnalyzer() 
    tokens = re.findall('[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+', text)
    with open("stopwords.txt", encoding="utf-8") as file: 
        stop_words = file.read()
    words = []
    for t in tokens:
        pv = morph.parse(t)
        if pv[0].tag.POS in imp_POS and pv[0].normal_form != 'быть' and \
        len(pv[0].normal_form) > 2 and pv[0].normal_form not in stop_words:
            words.append(pv[0].normal_form)
            
    text = ' '.join(words)
    if len(text) == 0:
        return [], {' ' : 0}
    counter=CountVectorizer(ngram_range=(1,1), token_pattern=r'[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+')
    res=counter.fit_transform([text])
    frq = sorted([(i, int(res[0][0,i])) for i in range(res[0].shape[1])], key = lambda x: x[1], reverse = True)[:10]
    vocab = {w:res[0,i] for w,i in counter.vocabulary_.items()}
    
    return [[w for w, n in counter.vocabulary_.items() if n==i][0] for i, f in frq], vocab

In [16]:
getMostFrequentWordsFiltered(train['text'][333])

(['адекватный',
  'документация',
  'коммуникабельность',
  'медицинский',
  'образование',
  'общение',
  'пользователь',
  'порядочность',
  'пунктуальность',
  'способность'],
 {'средний': array(1),
  'медицинский': array(1),
  'образование': array(1),
  'стрессоустойчивость': array(1),
  'коммуникабельность': array(1),
  'способность': array(1),
  'адекватный': array(1),
  'общение': array(1),
  'порядочность': array(1),
  'уверенный': array(1),
  'пользователь': array(1),
  'пунктуальность': array(1),
  'документация': array(1)})

In [11]:
# составим словарь и найдем 10 самых частотных слов
freqwords_train = []
vocabs_train = []

for i in tqdm(range(train.shape[0])):
    fw, v = getMostFrequentWordsFiltered(train['text'][i])
    freqwords_train.append(fw)
    vocabs_train.append(v)

100%|██████████| 31063/31063 [43:51<00:00, 12.40it/s] 


In [ ]:
train['Vocabular'] = vocabs_train
train['Freq Words'] = freqwords_train

In [ ]:
train.head(10)

In [17]:
# поиск самых частотных слов по всему словарю для пополнения списка стоп-слов
large_text = ' '
for i in tqdm(range(train.shape[0])):
    large_text += ' '.join(train['Vocabular'][i]) + ' '

  0%|          | 0/31063 [00:00<?, ?it/s]


KeyError: 'Vocabular'

In [15]:
print(large_text[:1000])

 развивающийся разработчик ведущий желательный технический английский язык высокий образование проектирование система документация разработка функциональность интеграция соответствие оптимизация производительность анализ ошибка сбой участие встреча бизнес заказчик профессиональный карьерный рост заработный программа внутренний обучение сертификация приём входящая звонок заявка обработка корзина почта переписка клиент консультирование ассортимент доставка телефон online-чат контроль отгрузка руководство интернет-магазин администрирование сайт наполнение изменение товар директ редактирование описание название приведение красивый понятный вид ведение сеть вконтакте покупатель шоурум удалённый стабильный заработный месяц оклад офис минута метро алексеевский испытательный срок федеральный сеть аптека связь активный расширение карьерный рост регион формат запрос разный клиент шанс карьер стабильный аптечный соответствие первое социальный льгота медицинский книжка сертификат счёт достойный бе

In [16]:
counter=CountVectorizer(ngram_range=(1,1), token_pattern=r'[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+')
counter.fit_transform([large_text])

<1x34754 sparse matrix of type '<class 'numpy.int64'>'
	with 34754 stored elements in Compressed Sparse Row format>

In [17]:
print("Vocabulary size: {}".format(len(counter.vocabulary_)))
print("Vocabulary content:\n {}".format(counter.vocabulary_))

Vocabulary size: 34754
Vocabulary content:
 {'развивающийся': 24477, 'разработчик': 24592, 'ведущий': 5115, 'желательный': 9670, 'технический': 30006, 'английский': 2474, 'язык': 34622, 'высокий': 6601, 'образование': 19044, 'проектирование': 23590, 'система': 27128, 'документация': 8919, 'разработка': 24588, 'функциональность': 32360, 'интеграция': 11168, 'соответствие': 27851, 'оптимизация': 19764, 'производительность': 23641, 'анализ': 2424, 'ошибка': 20451, 'сбой': 26405, 'участие': 31515, 'встреча': 6421, 'бизнес': 3874, 'заказчик': 9996, 'профессиональный': 23947, 'карьерный': 12158, 'рост': 25789, 'заработный': 10190, 'программа': 23470, 'внутренний': 5890, 'обучение': 19111, 'сертификация': 26917, 'приём': 23359, 'входящая': 6467, 'звонок': 10351, 'заявка': 10312, 'обработка': 19038, 'корзина': 13607, 'почта': 22695, 'переписка': 21042, 'клиент': 12616, 'консультирование': 13350, 'ассортимент': 2916, 'доставка': 9087, 'телефон': 29695, 'online-чат': 905, 'контроль': 13433, 'отг

In [18]:
# найдем десять самых важных слов по TFIDF для каждой вакансии
tfCounter = TfidfVectorizer(
    ngram_range=(1, 1),
    token_pattern=r'[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+')

tfres = tfCounter.fit_transform([' '.join(normalizePymorphy2(text)) for text in tqdm(train['text'])])

freqwords_tfidf_train = []

for i in tqdm(range(train.shape[0])):
    tfs = [(k, tfres[i][0, tfCounter.vocabulary_.get(k)]) for k in train.Vocabular.iloc[i] 
           if k in tfCounter.vocabulary_.keys()]
    fw = [w for w, f in sorted(tfs, key = lambda x: x[1], reverse = True)[:10]]
    freqwords_tfidf_train.append(fw)

100%|██████████| 31063/31063 [02:25<00:00, 213.93it/s]


In [19]:
train['Freq Words TFIDF'] = freqwords_tfidf_train

In [53]:
train.head()

,text,target,Vocabular,Freq Words,Freq Words TFIDF,Freq Words Tifdf_V_with_Params
id,,,,,,
0,<p>В молодой и дружный коллектив динамично раз...,0,"{'развивающийся': 2, 'разработчик': 2, 'ведущи...","[технический, английский, развивающийся, разра...","[развивающийся, разработчик, технический, англ...","[развивающийся, разработчик, сбой, функциональ..."
1,<p><strong>Обязанности: </strong>- Прием входя...,0,"{'приём': 1, 'входящая': 1, 'звонок': 1, 'заяв...","[заявка, клиент, месяц, сайт, товар, online-ча...","[online-чат, шоурум, сайт, корзина, вконтакте,...","[шоурум, корзина, вконтакте, приведение, дирек..."
2,<p><strong>Федеральная сеть аптек </strong>в с...,0,"{'федеральный': 1, 'сеть': 2, 'аптека': 3, 'св...","[аптека, аптечный, книжка, неделя, подходящий,...","[аптечный, аптека, подходящий, рассмотрение, т...","[аптечный, аптека, подходящий, рассмотрение, т..."
3,<strong>Обязанности:</strong> <ul> <li> <p>раз...,0,"{'разработка': 2, 'проектный': 2, 'документаци...","[автоматизация, документация, система, диспетч...","[автоматизация, диспетчеризация, раздел, систе...","[диспетчеризация, автоматизация, оздс, аскуэр,..."
4,<p>Производителю сантехники тм Domani Spa треб...,1,"{'производитель': 1, 'сантехник': 1, 'активный...","[аккуратность, активный, грузчик-комплектовщик...","[грузчик-комплектовщик, погрузка-выгрузка, ком...","[комплектовка, переулок, пунктуальный, сантехн..."


In [23]:
tifdf_with_parameters = TfidfVectorizer(smooth_idf=True, sublinear_tf=True, norm=None, analyzer='word')

tifdfwp = tifdf_with_parameters.fit_transform([' '.join(normalizePymorphy2(text)) for text in tqdm(train['text'])])

tifdf_with_parameters_list = []

for i in tqdm(range(train.shape[0])):
    tfs = [(k, tifdfwp[i][0, tifdf_with_parameters.vocabulary_.get(k)]) for k in train.Vocabular.iloc[i] 
           if k in tifdf_with_parameters.vocabulary_.keys()]
    fw = [w for w, f in sorted(tfs, key = lambda x: x[1], reverse = True)[:10]]
    tifdf_with_parameters_list.append(fw)

100%|██████████| 31063/31063 [02:27<00:00, 210.53it/s]


In [24]:
train['Freq Words Tifdf_V_with_Params'] = tifdf_with_parameters_list

In [72]:
train.head(20)

,text,target,Vocabular,Freq Words,Freq Words TFIDF,Freq Words Tifdf_V_with_Params
id,,,,,,
0,<p>В молодой и дружный коллектив динамично раз...,0,"{'развивающийся': 2, 'разработчик': 2, 'ведущи...","[технический, английский, развивающийся, разра...","[развивающийся, разработчик, технический, англ...","[развивающийся, разработчик, сбой, функциональ..."
1,<p><strong>Обязанности: </strong>- Прием входя...,0,"{'приём': 1, 'входящая': 1, 'звонок': 1, 'заяв...","[заявка, клиент, месяц, сайт, товар, online-ча...","[online-чат, шоурум, сайт, корзина, вконтакте,...","[шоурум, корзина, вконтакте, приведение, дирек..."
2,<p><strong>Федеральная сеть аптек </strong>в с...,0,"{'федеральный': 1, 'сеть': 2, 'аптека': 3, 'св...","[аптека, аптечный, книжка, неделя, подходящий,...","[аптечный, аптека, подходящий, рассмотрение, т...","[аптечный, аптека, подходящий, рассмотрение, т..."
3,<strong>Обязанности:</strong> <ul> <li> <p>раз...,0,"{'разработка': 2, 'проектный': 2, 'документаци...","[автоматизация, документация, система, диспетч...","[автоматизация, диспетчеризация, раздел, систе...","[диспетчеризация, автоматизация, оздс, аскуэр,..."
4,<p>Производителю сантехники тм Domani Spa треб...,1,"{'производитель': 1, 'сантехник': 1, 'активный...","[аккуратность, активный, грузчик-комплектовщик...","[грузчик-комплектовщик, погрузка-выгрузка, ком...","[комплектовка, переулок, пунктуальный, сантехн..."
5,<strong>Обязанности:</strong> <ul> <li>Активны...,0,"{'активный': 1, 'поиск': 2, 'клиент': 5, 'холо...","[клиент, продажа, контакт, навык, поиск, актив...","[клиент, продажа, контакт, понедельник-четверг...","[контакт, факс, продажа, клиент, поиск, сканер..."
6,<strong>Обязанности:</strong> <ul> <li>работа ...,1,"{'касса': 1, 'приготовление': 1, 'напиток': 1,...","[изделие, кондитерский, бесплатный, будний, вы...","[кондитерский, изделие, коктейль, будний, отда...","[кондитерский, изделие, коктейль, будний, отда..."
7,<strong>Обязанности:</strong> <p>-Ведение и св...,1,"{'ведение': 1, 'своевременный': 1, 'сдача': 1,...","[бухгалтерский, налоговый, аудиторский, аутсор...","[налоговый, союз, аутсорсинговый, тоо, бухгалт...","[союз, аутсорсинговый, тоо, налоговый, коррект..."
8,<p><strong>Обязанности</strong></p> <ul> <li>В...,0,"{'выявление': 1, 'проработка': 1, 'заказчик': ...","[разработка, заказчик, офис, английский, высок...","[заказчик, разработка, верхнеуровневой, it-тер...","[верхнеуровневой, вэбинар, учебник, шашка, при..."


## X_Train

In [75]:
X = np.array([' '.join((map(str, words))) for words in train['Freq Words Tifdf_V_with_Params']])
y = train['target']
X.shape

(31063,)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(21744,)
(21744,)
(9319,)
(9319,)


## Vectorize

In [114]:
# vectorizer = TfidfVectorizer(
#     sublinear_tf=True,
#     analyzer='word',
#     ngram_range=(1, 1),
#     max_features=30000)

vectorizer = TfidfVectorizer(
    ngram_range=(1, 1),
    token_pattern=r'[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+')

X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

## Models: Logistic Regression

## Parameter Tuning, Validate

In [80]:
losses = []
auc = []

classifier = LogisticRegression(solver='sag', C=10)

cv_loss = np.mean(cross_val_score(classifier, X_train_features, y_train, cv=5, scoring='neg_log_loss'))
losses.append(cv_loss)
print('CV Log_loss score is {}'.format(cv_loss))

cv_score = np.mean(cross_val_score(classifier, X_train_features, y_train, cv=5, scoring='accuracy'))
print('CV Accuracy score is {}'.format(cv_score))
    
classifier.fit(X_train_features, y_train)
y_pred = classifier.predict(X_test_features)
y_pred_prob = classifier.predict_proba(X_test_features)[:, 1]
auc_score = roc_auc_score(y_test, y_pred_prob)
auc.append(auc_score)
print("CV ROC_AUC score {}\n".format(auc_score))
    
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

CV Log_loss score is -0.18934754923344083
CV Accuracy score is 0.932579241323334
CV ROC_AUC score 0.9810119062643686

[[4681  261]
 [ 355 4022]]
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      4942
           1       0.94      0.92      0.93      4377

    accuracy                           0.93      9319
   macro avg       0.93      0.93      0.93      9319
weighted avg       0.93      0.93      0.93      9319



In [81]:
param_set1 = {
        'lr__C': [0.0001, 0.01, 0.1, 1 ,10, 100, 1000, 5000],
        'lr__penalty': ['l1', 'l2'],
        'lr__intercept_scaling': range(1, 10),
        'lr__class_weight': [None, 'balanced'],
        'lr__max_iter': range(1, 10),
        #'lr__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# param_set1 = {
#         'tree__max_depth': range(1, 20),
#         'tree__max_leaf_nodes': range(2, 20),
#         'tree__min_samples_split': range(2, 450, 10),
#         'tree__min_samples_leaf' : range(2, 1000, 10)
# }

# param_set1 = {
#     'knn__n_neighbors': range(1, 10)
# }

In [82]:
pipeline1 = Pipeline([
    ('lr', LogisticRegression())])

# pipeline1 = Pipeline([
#     ('tree', DecisionTreeClassifier())])

# pipeline1 = Pipeline([
#     ('knn', KNeighborsClassifier())])

X_features = vectorizer.fit_transform(X)
print(X_features.shape)

lr = LogisticRegression()
lr.fit(X_features, y);

(31063, 26231)


In [83]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

In [84]:
# hyper_search1 = RandomizedSearchCV(pipeline1, param_set1, n_iter=5, scoring='roc_auc', cv=cv,
#                                   n_jobs=2, refit=True, verbose=True, random_state=0)
# hyper_search1.fit(X, y);

In [85]:
# print(hyper_search1.best_params_)
# print(hyper_search1.best_score_)

In [86]:
# best_model_lr = hyper_search1.best_estimator_
# best_model_lr;

In [87]:
scores = cross_val_score(lr, X_features, y, cv=cv, scoring='roc_auc')
scores 
print(scores, sum(scores)/len(scores))

# TODO: локально очень высокая roc_auc, разобраться, почему

[0.98465554 0.98109223 0.98421123 0.98429799 0.98382358 0.98360364
 0.97845086 0.98196817 0.97992924 0.98353207] 0.9825564547057034


In [88]:
# from sklearn import metrics

# X_train, X_test, y_train, y_test = train_test_split(
#          X, y, test_size=0.3, random_state=0)

# lr.fit(X_train, y_train);

# #y_pred_proba1 = lr.predict_proba(X_test)[::,1]

# # fpr, tpr, _ = metrics.roc_auc_score(y_test, y_pred_proba1)
# # auc = metrics.roc_auc_score(y_test, y_pred_proba1)
# # plt.plot(fpr,tpr,label="logreg , auc="+str(auc))
# # plt.legend(loc=4)
# # plt.show()

# y_hat_test = lr.predict(X_test)
# y_hat_train = lr.predict(X_train)

# roc_auc_score(y_test, y_hat_test), roc_auc_score(y_train, y_hat_train)

## Fit

In [121]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 1),
    token_pattern=r'[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+')

X_features = vectorizer.fit_transform(X)
X_features.shape

(31063, 26231)

In [122]:
classifier = LogisticRegression()
    
classifier.fit(X_features, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

## Test Data Preprocessing

In [61]:
# составим словарь и найдем 10 самых частотных слов для обучающей выборки
freqwords_test = []
vocabs_test = []

for i in tqdm(range(test.shape[0])):
    fw, v = getMostFrequentWordsFiltered(test['text'][i])
    freqwords_test.append(fw)
    vocabs_test.append(v)


  2%|▏         | 583/31064 [00:55<41:27, 12.25it/s]


  4%|▍         | 1174/31064 [01:51<38:36, 12.90it/s]


  6%|▌         | 1761/31064 [02:44<46:39, 10.47it/s]


  8%|▊         | 2346/31064 [03:38<37:43, 12.69it/s]


  9%|▉         | 2914/31064 [21:26<46:36, 10.07it/s]


...

 93%|█████████▎| 28930/31064 [1:01:36<03:19, 10.68it/s]


 95%|█████████▍| 29485/31064 [1:02:28<02:08, 12.30it/s]


 97%|█████████▋| 30042/31064 [1:03:19<01:28, 11.54it/s]


 99%|█████████▊| 30604/31064 [1:04:12<00:36, 12.71it/s]


100%|██████████| 31064/31064 [1:04:54<00:00,  7.98it/s]

In [62]:
test['Vocabular'] = vocabs_test
test['Freq Words'] = freqwords_test

In [63]:
getMostFrequentWordsFiltered(test['text'][333])

(['контроль',
  'арендатор',
  'бизнес',
  'высокий',
  'грамотный',
  'деятельность',
  'зона',
  'качество',
  'клиниговый',
  'комплекс'],
 {'контроль': array(5),
  'зона': array(1),
  'координация': array(1),
  'качество': array(1),
  'объём': array(1),
  'подрядный': array(1),
  'соблюдение': array(1),
  'правило': array(1),
  'торговый': array(1),
  'комплекс': array(1),
  'арендатор': array(1),
  'осуществление': array(1),
  'рекламный': array(1),
  'маркетинговый': array(1),
  'деятельность': array(1),
  'клиниговый': array(1),
  'ресторанный': array(1),
  'бизнес': array(1),
  'преимущество': array(1),
  'высокий': array(1),
  'образование': array(1),
  'общий': array(1),
  'язык': array(1),
  'человек': array(1),
  'грамотный': array(1),
  'устный': array(1),
  'речь': array(1),
  'развивающийся': array(1),
  'официальный': array(1),
  'место': array(1),
  'метр': array(1),
  'пионерский': array(1),
  'сити': array(1),
  'молл': array(1)})

In [64]:
test.head()

,id,text,Vocabular,Freq Words
0,31063,<p><strong>В крупную компанию по организации и...,"{'приготовление': 1, 'питание': 2, 'бухгалтер'...","[ведение, общепит, питание, айко, бесплатный, ..."
1,31064,<p><strong>Обязанности:</strong></p> <ul> <li>...,"{'обеспечение': 2, 'необходимый': 2, 'функцион...","[бухгалтерский, конфигурация, обмен, пользоват..."
2,31065,<p> </p> <p><strong>Обязанности:</strong></p> ...,"{'отгрузка': 1, 'приём': 1, 'товар': 2, 'склад...","[склад, товар, ведение, документация, доставка..."
3,31066,<p><strong>Обязанности:</strong></p> <ul> <li>...,"{'приготовление': 1, 'холодное': 1, 'горячее':...","[блюдо, горячее, книжка, меню, наличие, обеспе..."
4,31067,<p><strong>Вакансия СРОЧНАЯ!</strong></p> <p><...,"{'срочный': 1, 'внимание': 1, 'просьба': 1, 'т...","[руководство, бухгалтер, оборудование, бухгалт..."


In [65]:
tifdf_with_parameters = TfidfVectorizer(smooth_idf=True, sublinear_tf=True, norm=None, analyzer='word')

tifdfwp_test = tifdf_with_parameters.fit_transform([' '.join(normalizePymorphy2(text)) for text in tqdm(test['text'])])

tifdf_with_parameters_list_test = []

for i in tqdm(range(test.shape[0])):
    tfs = [(k, tifdfwp_test[i][0, tifdf_with_parameters.vocabulary_.get(k)]) for k in test.Vocabular.iloc[i] 
           if k in tifdf_with_parameters.vocabulary_.keys()]
    fw = [w for w, f in sorted(tfs, key = lambda x: x[1], reverse = True)[:10]]
    tifdf_with_parameters_list_test.append(fw)


  4%|▍         | 1305/31064 [00:39<14:48, 33.48it/s]


  9%|▊         | 2648/31064 [01:20<15:49, 29.94it/s]


 13%|█▎        | 3982/31064 [01:59<11:26, 39.47it/s]


 17%|█▋        | 5258/31064 [02:39<11:46, 36.52it/s]


 21%|██        | 6578/31064 [03:19<11:50, 34.48it/s]


...

 23%|██▎       | 7045/31064 [00:33<01:59, 200.80it/s]


 44%|████▍     | 13645/31064 [01:04<01:14, 234.16it/s]


 65%|██████▌   | 20282/31064 [01:34<00:50, 211.49it/s]


 86%|████████▋ | 26797/31064 [02:05<00:19, 214.52it/s]


100%|██████████| 31064/31064 [02:25<00:00, 213.53it/s]

In [66]:
test['Freq Words Tifdf_V_with_Params'] = tifdf_with_parameters_list_test

In [70]:
test.head(20)

,id,text,Vocabular,Freq Words,Freq Words Tifdf_V_with_Params
0,31063,<p><strong>В крупную компанию по организации и...,"{'приготовление': 1, 'питание': 2, 'бухгалтер'...","[ведение, общепит, питание, айко, бесплатный, ...","[калькулятор, общепит, айко, советский, бухгал..."
1,31064,<p><strong>Обязанности:</strong></p> <ul> <li>...,"{'обеспечение': 2, 'необходимый': 2, 'функцион...","[бухгалтерский, конфигурация, обмен, пользоват...","[обмен, конфигурация, обновление, функционал, ..."
2,31065,<p> </p> <p><strong>Обязанности:</strong></p> ...,"{'отгрузка': 1, 'приём': 1, 'товар': 2, 'склад...","[склад, товар, ведение, документация, доставка...","[елизаровский, склад, дыбенко, пролетарский, о..."
3,31066,<p><strong>Обязанности:</strong></p> <ul> <li>...,"{'приготовление': 1, 'холодное': 1, 'горячее':...","[блюдо, горячее, книжка, меню, наличие, обеспе...","[скользящий, горячее, холодное, меню, одежда, ..."
4,31067,<p><strong>Вакансия СРОЧНАЯ!</strong></p> <p><...,"{'срочный': 1, 'внимание': 1, 'просьба': 1, 'т...","[руководство, бухгалтер, оборудование, бухгалт...","[пражский, вход, балтавтотрейд, послый, крыльц..."
5,31068,<p><strong>Функции: </strong></p> <ul> <li>Ана...,"{'функция': 1, 'анализ': 2, 'контроль': 1, 'ка...","[навык, анализ, аналитика, интересный, использ...","[автф, хранилище, физтех, физфак, нгт, николае..."
6,31069,<p>Туроператору Планета Travel требуются менед...,"{'туроператор': 1, 'планет': 1, 'менеджер': 1,...","[заявка, зимний, направление, продажа, сезон, ...","[мальдивы, зимний, оаэ, сезон, возвращение, ту..."
7,31070,<p><strong>В большую семью сети &quot;Т-кафе&q...,"{'большой': 1, 'сеть': 1, 'том-кафе': 1, 'пост...","[алкогольный, бармен, бесплатный, больничный, ...","[партизан, красных, кафе, тяжёлый, отчисление,..."
8,31071,<p><strong>Stream</strong> – дочерняя компания...,"{'дочерний': 1, 'мтс': 4, 'задача': 1, 'мобиль...","[мобильный, мтс, проект, врач, жизнь, компенса...","[мтс, медиатека, превращение, мобильный, телем..."
9,31072,<p>В нашей компании имеется СВОЯ КРУГЛОСУТОЧНА...,"{'круглосуточный': 2, 'диспетчерский': 2, 'зак...","[автомобиль, вождение, диспетчерский, заказ, к...","[диспетчерский, круглосуточный, автомобиль, но..."


## X_Test

In [123]:
X_m_test = np.array([' '.join((map(str, words))) for words in test['Freq Words Tifdf_V_with_Params']])
X_m_test.shape, 
X_m_test[19]

'делопроизводство кадровый розничный касса банк сверка кофейня отчёт кредиторский ведение'

## Vectorize

In [124]:
X_m_test_features = vectorizer.transform(X_m_test)
X_m_test_features.shape

(31064, 26231)

## Predict

In [125]:
pred = classifier.predict_proba(X_m_test_features)[:,1]

## Submit

In [126]:
pd.DataFrame(zip(test['id'], pred), columns = ['id','target']).to_csv('pred_hw8.csv', sep=',', index=False)